In [1]:
%cd /home/q123/Desktop/explo

### local imports 
from src.environment import EnvironmentObjective
from src.optim import step
from src.policy import MLP

### botorch
from botorch.fit import fit_gpytorch_model
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood

from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf

### general imports
import torch
import gym
import logging

logging.basicConfig(filename='myapp.log', level=logging.INFO)

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/explo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

### initialize policy
mlp = MLP([3,1])

# Initialize environment

objective_env = EnvironmentObjective(
  env=gym.make("Pendulum-v1"),
  mlp=mlp,
  manipulate_state=None,
  manipulate_reward=None,
)

### initialize train_x, train_y
train_x = torch.rand(1,mlp.len_params) ## [n_trials,n_params]
train_y = [objective_env.run(p)[0] for p in train_x]
train_y = torch.Tensor(train_y).reshape(-1,1)  ## [n_trials,1]

### initialize model
model = SingleTaskGP(train_X=train_x, train_Y=train_y)

### now we loop :
max_iter = 1

for i in range(max_iter):

  step(model,objective_env)
  print("outputscale",model.covar_module.raw_outputscale)

  if i % 100 == 0:

    best_val = model.train_targets.max()
    curr_val = model.train_targets[-1]
    print(f'curr {curr_val} max {best_val}')

outputscale Parameter containing:
tensor(409.8265, requires_grad=True)
curr -1500.7691650390625 max -1500.7691650390625
